In [71]:
from pytube import YouTube
import os
import requests
from pydub import AudioSegment
import openai
from typing import Final
from dotenv import load_dotenv

# STEP 0: load our token from somwhere safe
load_dotenv()
OPEN_API_TOKEN: Final[str] = os.getenv('OPEN_API_TOKEN')
openai.api_key = OPEN_API_TOKEN


In [3]:
link = "https://www.youtube.com/watch?v=IHQr0HCIN2w"

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'age_restricted',
 'author',
 'bypass_age_gate',
 'caption_tracks',
 'captions',
 'channel_id',
 'channel_url',
 'check_availability',
 'description',
 'embed_html',
 'fmt_streams',
 'from_id',
 'initial_data',
 'js',
 'js_url',
 'keywords',
 'length',
 'metadata',
 'publish_date',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'streaming_data',
 'streams',
 'thumbnail_url',
 'title',
 'vid_info',
 'views',
 'watch_html']

In [31]:
def get_mp4_from_youtube_link(link):
    youtube_object = YouTube(link)
    print(youtube_object)
    youtube_object.title
    youtube_object.thumbnail_url
    #yt.streams.filter(file_extension='mp4')
    #save_video_path = "G:/temp/youtube-videos/"
    output_file_name = youtube_object.title + ".mp4"
    #download in local file system
    youtube_object.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first().download(filename = output_file_name)
    return output_file_name
    

#! consider temp file https://www.geeksforgeeks.org/python-tempfile-module/

In [64]:
#get video
output_file_name = get_mp4_from_youtube_link(link)

def does_file_exist(output_file_name):
    return os.path.isfile(output_file_name)


<pytube.__main__.YouTube object: videoId=IHQr0HCIN2w>


In [52]:
def get_audio_from_youtube_link(link):
    youtube_object = YouTube(link)
    print(youtube_object)
    youtube_object.title
    youtube_object.thumbnail_url
    #download in local file system
    youtube_object.streams.filter(only_audio=True).first().download() 
    output_file_name = youtube_object.title + "." + "mp4"
    #youtube_object.streams.filter(progressive=True, file_extension=extension).order_by('resolution').desc().first().download(filename = output_file_name)
    return output_file_name

#! consider temp file https://www.geeksforgeeks.org/python-tempfile-module/

In [53]:
output_file_name = get_audio_from_youtube_link(link)
does_file_exist(output_file_name)

<pytube.__main__.YouTube object: videoId=IHQr0HCIN2w>


True

In [58]:
def segment_and_calculate_volume(audio_file, segment_length_ms):
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)

    # Calculate the total number of segments
    num_segments = len(audio) // segment_length_ms

    # Initialize a list to store the speech volumes of each segment
    segment_volumes = []

    # Iterate over the audio and segment it
    for i in range(num_segments):
        start_time = i * segment_length_ms
        end_time = (i + 1) * segment_length_ms
        segment = audio[start_time:end_time]

        # Extract just the speech portion (assuming it's in a certain range of frequencies)
        speech_segment = segment.low_pass_filter(3000)

        # Calculate the average volume level in decibels (dB)
        speech_volume = speech_segment.dBFS

        segment_volumes.append(speech_volume)

    return segment_volumes


In [62]:

def calculate_average_speech_volume(audio_file):
    "average audio volume"
    # Load the audio file
    audio = AudioSegment.from_file(audio_file)

    # Extract just the speech portion (assuming it's in a certain range of frequencies)
    speech = audio.low_pass_filter(3000)

    # Calculate the average volume level in decibels (dB)
    speech_volume = speech.dBFS

    return speech_volume


In [63]:
audio_file = output_file_name #path to the audio file
avg_speech_volume = calculate_average_speech_volume(audio_file)
print("Avg file Speech volume (dB):", avg_speech_volume)

segment_length_ms = 5000  # Length of each segment in milliseconds (adjust as needed)
segment_volumes = segment_and_calculate_volume(audio_file, segment_length_ms)
print("Speech volumes of each segment (dB):", segment_volumes)


Avg file Speech volume (dB): -19.250769694864182
Speech volumes of each segment (dB): [-18.472130474698666, -18.828373344639978, -17.71265477882404, -20.47336318751104, -22.86180704870788, -22.08334043884667, -21.588927985796428, -18.174923872520875, -21.553987442786596, -23.269209590481715, -26.758268700597117, -19.489902520671702, -22.832375797717752, -22.006191655276904, -20.645814504854766, -24.900080538835105, -20.42034072466638, -21.59529594036176, -19.063141410064862, -19.507416923110906, -22.185010225928103, -23.543854094269246, -20.490174592058622, -20.729016065727212, -20.434262653657562, -20.334519284236265, -20.318006181091377, -23.085778795293837, -23.476351983633272, -16.633653470688124, -17.503346105260743, -21.331781787045543, -15.825120308907765, -18.239615504517584, -20.389790801560256, -33.18650981434835, -22.009531739777994, -19.420195952840512, -24.807380730057183, -37.053842065562876, -11.988346495422968, -9.67598282816908, -47.89752007507736, -32.225511333511086,

In [73]:


audio_file = open(output_file_name, "rb")
#openai.OpenAI() below?
transcript = openai.audio.transcriptions.create(
  file=audio_file,
  model="whisper-1",
  response_format="verbose_json",
  timestamp_granularities=["word"]
)

print(transcript.words)

APIStatusError: Error code: 413 - {'error': {'message': 'Maximum content size limit (26214400) exceeded (26216694 bytes read)', 'type': 'server_error', 'param': None, 'code': None}}

In [35]:
#some code for emotion from hume
# url = "https://api.hume.ai/v0/batch/jobs"

# payload = "{\"models\":{\"face\":{\"fps_pred\":3,\"prob_threshold\":0.99,\"identify_faces\":false,\"min_face_size\":60,\"save_faces\":false},\"prosody\":{\"granularity\":\"utterance\",\"window\":{\"length\":4,\"step\":1}},\"language\":{\"granularity\":\"word\"}},\"transcription\":{\"language\":null,\"identify_speakers\":false,\"confidence_threshold\":0.5},\"notify\":false}"
# headers = {
#     "accept": "application/json; charset=utf-8",
#     "content-type": "application/json; charset=utf-8",
#     "X-Hume-Api-Key": "YOUR API KEY HERE"
# }

# response = requests.post(url, data=payload, headers=headers)

# print(response.text)

True


1. get youtube mp4
2. get subtitles - enhanced input
3. convert youtube to anime
4. maybe black and white it
5. get screenshots every 5 seconds + subtitle for timestamp
6. put subtitles onto screenshot, in speechbubble? to the right person? (can hard code it, like stage it - switch sides whenever person speaking)
7. format into comic panels
8. compile publish as pdf

video - popular